In [205]:
import sys
sys.path.append('../../../src/')

import os
import pandas as pd
import numpy as np
import config
import utils
import torch
import pickle
import train_impf_MLP

from Model import Impf_DNAMLP
from torch.nn.functional import softmax
from utils import impf_make_ndarray_from_csv, get_int_label
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from Dataset import CNS

In [206]:
inner_fold = '3.4'
outer_fold = f"{fold.split('.')[0]}.0"
group = 'Sella'
alg = 'RF'
print(outer_fold)

3.0


In [207]:
dat_cfg = config.data_config
clf_cfg = config.classifier_config
impf_cfg = config.impf_config
TRAIN_CSV_DIR = dat_cfg['TRAIN_CSV_DIR']
TEST_CSV_DIR = dat_cfg['TEST_CSV_DIR']

INNER_FOLD_CSV = os.path.join(TEST_CSV_DIR, group, f'{inner_fold}_test.csv')
OUTER_FOLD_CSV = os.path.join(TEST_CSV_DIR, group, f'{outer_fold}_test.csv')
device = impf_cfg['device']
device

'cuda'

In [208]:
with open(os.path.join(impf_cfg['IMPORTANT_FEATURES_DIR'], alg, group, f'{outer_fold}_combined.pkl'), 'rb') as handle:
    impf = pickle.load(handle)
print(len(impf))

2049


In [209]:
impf

{'cg12406391',
 'cg01817364',
 'cg04895288',
 'cg23233416',
 'cg05788681',
 'cg22274196',
 'cg07810039',
 'cg26668919',
 'cg07690455',
 'cg12719030',
 'cg09217215',
 'cg08835113',
 'cg13278738',
 'cg17343563',
 'cg25809561',
 'cg26396492',
 'cg16672562',
 'cg23878831',
 'cg04619304',
 'cg09977852',
 'cg01064265',
 'cg10528424',
 'cg10194729',
 'cg15988569',
 'cg18864227',
 'cg22856324',
 'cg23704802',
 'cg05740106',
 'cg15934004',
 'cg04466840',
 'cg07015803',
 'cg09140778',
 'cg22953237',
 'cg13947995',
 'cg02340083',
 'cg22984571',
 'cg19594666',
 'cg25503149',
 'cg19383430',
 'cg20175702',
 'cg22982368',
 'cg10715092',
 'cg14189678',
 'cg07921384',
 'cg08124030',
 'cg09639151',
 'cg20377305',
 'cg12785342',
 'cg14691529',
 'cg19656282',
 'cg08557179',
 'cg01366378',
 'cg04640920',
 'cg26978172',
 'cg12236164',
 'cg25176883',
 'cg23270712',
 'cg13945749',
 'cg03591606',
 'cg22489321',
 'cg12276123',
 'cg08781140',
 'cg06679334',
 'cg00848945',
 'cg07537417',
 'cg20277282',
 'cg029400

In [210]:
df_inner = pd.read_csv(INNER_FOLD_CSV, index_col=0)
df_outer = pd.read_csv(OUTER_FOLD_CSV, index_col=0)
df_inner.shape, df_outer.shape

((42, 10002), (59, 10002))

In [211]:
pd.Series(pd.read_csv(OUTER_FOLD_CSV, index_col=0).iloc[:, -1]).value_counts()

Sella      34
Control    25
Name: class_group, dtype: int64

## Load random test sample and predict

In [224]:
rand = np.random.randint(df_outer.shape[0])
sample = df_outer.iloc[rand, :]
rand, sample

(48,
 cg22054918        0.066219
 cg16476975        0.139864
 cg25570913        0.048185
 cg03940848        0.878975
 cg02809746        0.015684
                   ...     
 cg22513691        0.889916
 cg00743629         0.72401
 cg13621317        0.889066
 label          PITAD, ACTH
 class_group          Sella
 Name: 3998920095_R05C02, Length: 10002, dtype: object)

In [226]:
sample = df_outer.loc[:, list(impf)].iloc[rand,:]
sample, sample.shape

(cg12406391    0.464937
 cg01817364    0.145447
 cg04895288    0.291135
 cg23233416    0.873583
 cg05788681    0.629531
                 ...   
 cg17184704    0.826883
 cg05042749    0.671590
 cg09393254    0.137970
 cg24675344    0.229020
 cg25356006    0.905762
 Name: 3998920095_R05C02, Length: 2049, dtype: float64,
 (2049,))

In [229]:
sample_features = np.expand_dims(np.array(sample), axis = 0)
sample_features
# sample_features, type(sample_features), sample_features.shape, type(sample_features[0][0])

array([[0.46493739, 0.14544733, 0.29113533, ..., 0.13796978, 0.22902013,
        0.90576227]])

In [230]:
IMPF_MLP_STATE_FILE = os.path.join(impf_cfg['MLP_BEST_STATES_DIR'], alg, group, f'{fold}.pth')

n_classes = impf_cfg['n_classes']
in_features = len(impf)

model = Impf_DNAMLP(in_features, n_classes)
model.load_state_dict(torch.load(IMPF_MLP_STATE_FILE))
model.to(device)

Impf_DNAMLP(
  (densenet): Sequential(
    (0): Linear(in_features=2049, out_features=2, bias=True)
  )
)

In [231]:
sample_features.shape

(1, 2049)

In [232]:
probs = softmax(model((torch.Tensor(sample_features).to(device))), dim = 1)
probs

tensor([[0.0525, 0.9475]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [221]:
epoch = 1
test_features, test_labels = impf_make_ndarray_from_csv(group, outer_fold, impf, mode = 'test')
test_labels_int = np.array([get_int_label(label, group) for label in test_labels])
test_dataset = CNS(test_features, test_labels_int, mode = 'val')
test_loader = DataLoader(test_dataset, batch_size = impf_cfg['mlp_val_batch_size'], shuffle = False)
criterion = CrossEntropyLoss()
# optimizer = Adam(model.parameters(), lr = impf_cfg['mlp_lr'], weight_decay = impf_cfg['mlp_weight_decay'])
res = train_impf_MLP.impf_val_epoch(epoch, model, test_loader, criterion, impf_cfg['device'])
test_loss, test_acc, test_me, test_bs, test_auc, test_f1, test_precision, test_recall, test_cfs = res

../../../src/utils.py:230: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  test_features = np.array(df_test.loc[:,impf])


In [222]:
res

(1.3190172463655472,
 42.3728813559322,
 57.6271186440678,
 0.9211761186893705,
 0.5,
 0.5952380952380952,
 0.423728813559322,
 1.0,
 array([[ 0, 34],
        [ 0, 25]]))

In [172]:
train_features, train_labels = impf_make_ndarray_from_csv(group, fold, impf, mode = 'train')
val_features, val_labels = impf_make_ndarray_from_csv(group, fold, impf, mode = 'test')
test_features, test_labels = impf_make_ndarray_from_csv(group, outer_fold, impf, mode = 'test')

train_labels_int = np.array([get_int_label(label, group) for label in train_labels])
val_labels_int = np.array([get_int_label(label, group) for label in val_labels])
test_labels_int = np.array([get_int_label(label, group) for label in test_labels])

train_dataset = CNS(train_features, train_labels_int, mode = 'train')
val_dataset = CNS(val_features, val_labels_int, mode = 'val')
test_dataset = CNS(test_features, test_labels_int, mode = 'val')
train_loader = DataLoader(train_dataset, batch_size = impf_cfg['mlp_train_batch_size'], shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = impf_cfg['mlp_val_batch_size'], shuffle = False)
test_loader = DataLoader(test_dataset, batch_size = impf_cfg['mlp_val_batch_size'], shuffle = False)

# Init model object
in_features = len(impf)
if group in utils.low_perf_groups:
    model = Impf_GlioMLP(in_features, impf_cfg['n_classes'])
else:
    model = Impf_DNAMLP(in_features, impf_cfg['n_classes'])

../../../src/utils.py:222: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  train_features = np.array(df_train.loc[:,impf])
../../../src/utils.py:230: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  test_features = np.array(df_test.loc[:,impf])
../../../src/utils.py:230: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  test_features = np.array(df_test.loc[:,impf])


In [173]:
%%capture
all_res = {}
for fold in utils.inner_folds:
    res = train_impf_MLP.impf_run(group, alg, fold, train_loader, val_loader, test_loader, model, criterion, optimizer, impf_cfg, 'no_save')
    all_res[f'{fold}'] = res

In [174]:
all_res

{'1.1': {'train_loss:': 0.785927942243673,
  'train_accs': 27.564102564102566,
  'train_me': 72.43589743589743,
  'train_bs': 0.591236758393547,
  'train_precision': 0.18048780487804877,
  'train_recall': 0.961038961038961,
  'train_auc': 0.5508519613378948,
  'train_f1': 0.30390143737166325,
  'train_cfs': array([[ 55, 336],
         [  3,  74]]),
  'val_loss:': 0.7868401237896511,
  'val_accs': 27.027027027027028,
  'val_me': 72.97297297297297,
  'val_bs': 0.592011551446238,
  'val_precision': 0.16666666666666666,
  'val_recall': 0.9411764705882353,
  'val_auc': 0.5450563204005006,
  'val_f1': 0.2831858407079646,
  'val_cfs': array([[14, 80],
         [ 1, 16]]),
  'test_loss:': 0.7685392320156097,
  'test_accs': 29.931972789115648,
  'test_me': 70.06802721088435,
  'test_bs': 0.5742924522614892,
  'test_precision': 0.1953125,
  'test_recall': 1.0,
  'test_auc': 0.5778688524590163,
  'test_f1': 0.32679738562091504,
  'test_cfs': array([[ 19, 103],
         [  0,  25]])},
 '1.2': {'tr